In [1]:
import shutil
import os

# Delete HateDeRC directory if it exists
if os.path.exists('HateDeRC'):
  shutil.rmtree('HateDeRC')
!git clone https://github.com/jamesalv/HateDeRC
%cd HateDeRC

Cloning into 'HateDeRC'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 45 (delta 19), reused 29 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 4.59 MiB | 10.21 MiB/s, done.
Resolving deltas: 100% (19/19), done.
/content/HateDeRC


In [2]:
from TrainingConfig import TrainingConfig
from typing import Dict, Any, Tuple, List
import numpy as np
import torch
from transformers import AutoTokenizer
import json

In [3]:
data_path = 'Data/dataset.json'

In [4]:
config = TrainingConfig()

In [5]:
# Seed all randomness for reproducibility
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(config.seed)
if device.type == 'cuda':
    torch.cuda.manual_seed_all(config.seed)
np.random.seed(config.seed)

# Preprocessing

In [6]:
import re
import string

def deobfuscate_text(text):
    """
    Normalize common text obfuscation patterns to reveal original words.
    Useful for hate speech detection and content analysis.

    Args:
        text (str): Input text with potential obfuscations

    Returns:
        str: Text with obfuscations normalized
    """
    if not isinstance(text, str):
        return text

    # Make a copy to work with
    result = text.lower()

    # 1. Handle asterisk/symbol replacements
    symbol_patterns = {
        # Common profanity
        r'f\*+c?k': 'fuck',
        r'f\*+': 'fuck',
        r's\*+t': 'shit',
        r'b\*+ch': 'bitch',
        r'a\*+s': 'ass',
        r'd\*+n': 'damn',
        r'h\*+l': 'hell',
        r'c\*+p': 'crap',

        # Slurs and hate speech terms (be comprehensive for detection)
        r'n\*+g+[aer]+': 'nigger',  # Various n-word obfuscations
        r'f\*+g+[ot]*': 'faggot',
        r'r\*+[dt]ard': 'retard',
        r'sp\*+c': 'spic',

        # Other symbols
        r'@ss': 'ass',
        r'b@tch': 'bitch',
        r'sh!t': 'shit',
        r'f#ck': 'fuck',
        r'd@mn': 'damn',
    }

    for pattern, replacement in symbol_patterns.items():
        result = re.sub(pattern, replacement, result, flags=re.IGNORECASE)

    # 2. Handle character spacing (f u c k -> fuck)
    spacing_patterns = {
        r'\bf\s+u\s+c\s+k\b': 'fuck',
        r'\bs\s+h\s+i\s+t\b': 'shit',
        r'\bd\s+a\s+m\s+n\b': 'damn',
        r'\bh\s+e\s+l\s+l\b': 'hell',
        r'\ba\s+s\s+s\b': 'ass',
        r'\bc\s+r\s+a\s+p\b': 'crap',
    }

    for pattern, replacement in spacing_patterns.items():
        result = re.sub(pattern, replacement, result, flags=re.IGNORECASE)

    # 3. Handle number/letter substitutions
    leet_patterns = {
        # Basic leet speak
        r'\b3\s*1\s*1\s*3\b': 'elle',  # 3113 -> elle
        r'\bf4g\b': 'fag',
        r'\bf4gg0t\b': 'faggot',
        r'\bn00b\b': 'noob',
        r'\bl33t\b': 'leet',
        r'\bh4t3\b': 'hate',
        r'\b5h1t\b': 'shit',
        r'\bf0ck\b': 'fock',
    }

    for pattern, replacement in leet_patterns.items():
        result = re.sub(pattern, replacement, result, flags=re.IGNORECASE)

    # 4. Handle repeated characters and separators
    # Remove excessive punctuation between letters
    result = re.sub(r'([a-z])[^\w\s]+([a-z])', r'\1\2', result)

    # Handle underscore separation
    result = re.sub(r'([a-z])_+([a-z])', r'\1\2', result)

    # Handle dot separation
    result = re.sub(r'([a-z])\.+([a-z])', r'\1\2', result)

    # 5. Handle common misspellings/variations used for evasion
    evasion_patterns = {
        r'\bfuk\b': 'fuck',
        r'\bfuq\b': 'fuck',
        r'\bfck\b': 'fuck',
        r'\bshyt\b': 'shit',
        r'\bshit\b': 'shit',
        r'\bbiatch\b': 'bitch',
        r'\bbeatch\b': 'bitch',
        r'\basshole\b': 'asshole',
        r'\ba55hole\b': 'asshole',
        r'\btard\b': 'retard',
        r'\bfagg\b': 'fag',
    }

    for pattern, replacement in evasion_patterns.items():
        result = re.sub(pattern, replacement, result, flags=re.IGNORECASE)

    # 6. Clean up multiple spaces
    result = re.sub(r'\s+', ' ', result).strip()

    return result

In [7]:
def aggregate_rationales(rationales, labels, post_length, drop_abnormal=False):
    """
    If all 3 annotators are normal → 3 zero spans → average (all zeros).
    If k annotators are non-normal and k spans exist → average the k spans (no added zeros).
    If k non-normal but fewer than k spans:
        If the missing annotators are non-normal → do not fill with zeros; average only existing spans and record rationale_support = #spans.
        If the missing annotators are normal (e.g., 2 hate + 1 normal + 2 spans) → append one zero span for the normal.
    """
    count_normal = labels.count(0)
    count_hate = labels.count(1)
    count_rationales = len(rationales)
    pad = np.zeros(post_length, dtype="int").tolist()

    # If there are hate labels but no rationales, something is wrong
    if count_hate > 0 and count_rationales == 0:
        if drop_abnormal:
            return None

        # Else just fill with 0
        return np.zeros(post_length).tolist()

    # If all annotators are normal, return all zeros
    if count_normal == 3:
        return np.zeros(post_length).tolist()

    # If we have hate annotators
    if count_hate > 0:
        # Case 1: Number of rationales matches number of hate annotators
        if count_rationales == count_hate:
            return np.average(rationales, axis=0).tolist()

        # Case 2: Fewer rationales than hate annotators
        elif count_rationales < count_hate:
            # Add zero padding for normal annotators only
            rationales_copy = rationales.copy()
            zeros_to_add = count_normal
            for _ in range(zeros_to_add):
                rationales_copy.append(pad)
            return np.average(rationales_copy, axis=0).tolist()

        # Case 3: More rationales than hate annotators (shouldn't happen normally)
        else:
            # Just average what we have
            return np.average(rationales, axis=0).tolist()

    # Fallback: return zeros if no clear case matches
    return np.zeros(post_length).tolist()

In [8]:
from typing import List, Tuple

def preprocess_text(raw_text):
    preprocessed_text = raw_text
    # # Remove HTML tags <>
    preprocessed_text = preprocessed_text.replace("<", "").replace(">", "")
    # # De-Obsfucate Patterns
    preprocessed_text = deobfuscate_text(preprocessed_text)

    return preprocessed_text


def create_text_segment(
    text_tokens: List[str], rationale_mask: List[int]
) -> List[Tuple[List[str], int]]:
    """
    Process a rationale mask to identify contiguous segments of highlighted text.
    Then create a segmented representation of the tokens

    Args:
        text_tokens: Original text tokens
        mask: Binary mask where 1 indicates a highlighted token (this consists of mask from 3 annotators)

    Returns:
        A list of tuples (text segment, mask value)
    """
    # Handle case where mask is empty (no rationale provided), usually this is normal classification
    mask = rationale_mask

    # for mask in all_rationale_mask:
    # Find breakpoints (transitions between highlighted/1 and non-highlighted/0)
    breakpoints = []
    mask_values = []

    # Always start with position 0
    breakpoints.append(0)
    mask_values.append(mask[0])

    # Find transitions in the mask
    for i in range(1, len(mask)):
        if mask[i] != mask[i - 1]:
            breakpoints.append(i)
            mask_values.append(mask[i])

    # Always end with the length of the text
    if breakpoints[-1] != len(mask):
        breakpoints.append(len(mask))

    # Create segments based on breakpoints
    segments = []
    for i in range(len(breakpoints) - 1):
        start = breakpoints[i]
        end = breakpoints[i + 1]
        segments.append((text_tokens[start:end], mask_values[i]))

    return segments


def align_rationales(tokens, rationales, tokenizer, max_length=128):
    """
    Align rationales with tokenized text while handling different tokenizer formats.

    Args:
        tokens: Original text tokens
        rationales: Original rationale masks
        tokenizer: The tokenizer to use
        max_length: Maximum sequence length

    Returns:
        Dictionary with tokenized inputs and aligned rationale masks
    """
    segments = create_text_segment(tokens, rationales)
    all_human_rationales = []
    all_input_ids = []
    all_attention_mask = []
    all_token_type_ids = []
    all_rationales = []
    for text_segment, rationale_value in segments:
        inputs = {}
        concatenated_text = " ".join(text_segment)
        processed_segment = preprocess_text(concatenated_text)
        tokenized = tokenizer(
            processed_segment, add_special_tokens=False, return_tensors="pt"
        )

        # Extract the relevant data
        segment_input_ids = tokenized["input_ids"][0]
        segment_attention_mask = tokenized["attention_mask"][0]
        # Handle token_type_ids if present
        if "token_type_ids" in tokenized:
            segment_token_type_ids = tokenized["token_type_ids"][0]
            all_token_type_ids.extend(segment_token_type_ids)

        # Add input IDs and attention mask
        all_input_ids.extend(segment_input_ids)
        all_attention_mask.extend(segment_attention_mask)

        # Add rationales (excluding special tokens)
        segment_rationales = [rationale_value] * len(segment_input_ids)
        all_rationales.extend(segment_rationales)
    # Get special token IDs
    cls_token_id = tokenizer.cls_token_id
    sep_token_id = tokenizer.sep_token_id

    # Add special tokens at the beginning and end
    all_input_ids = [cls_token_id] + all_input_ids + [sep_token_id]
    all_attention_mask = [1] + all_attention_mask + [1]

    # Handle token_type_ids if the model requires it
    if hasattr(tokenizer, "create_token_type_ids_from_sequences"):
        all_token_type_ids = tokenizer.create_token_type_ids_from_sequences(
            all_input_ids[1:-1]
        )
    elif all_token_type_ids:
        all_token_type_ids = [0] + all_token_type_ids + [0]
    else:
        all_token_type_ids = [0] * len(all_input_ids)

    # Check tokenized vs rationales length
    if len(all_input_ids) != len(all_attention_mask):
        print("Warning: length of tokens and rationales do not match")

    # Add zero rationale values for special tokens
    all_rationales = [0] + all_rationales + [0]

    # Truncate to max length if needed
    if len(all_input_ids) > max_length:
        print("WARNING: NEED TO TRUNCATE")
        all_input_ids = all_input_ids[:max_length]
        all_attention_mask = all_attention_mask[:max_length]
        all_token_type_ids = all_token_type_ids[:max_length]
        all_rationales = all_rationales[:max_length]

    # Pad to max_length if needed
    pad_token_id = tokenizer.pad_token_id
    padding_length = max_length - len(all_input_ids)

    if padding_length > 0:
        all_input_ids = all_input_ids + [pad_token_id] * padding_length
        all_attention_mask = all_attention_mask + [0] * padding_length
        all_token_type_ids = all_token_type_ids + [0] * padding_length
        all_rationales = all_rationales + [0] * padding_length

    # Convert lists to tensors
    inputs = {
        "input_ids": torch.tensor([all_input_ids], dtype=torch.long),
        "attention_mask": torch.tensor([all_attention_mask], dtype=torch.long),
        "token_type_ids": (
            torch.tensor([all_token_type_ids], dtype=torch.long)
            if "token_type_ids" in tokenizer.model_input_names
            else None
        ),
        "rationales": torch.tensor([all_rationales], dtype=torch.float32),
    }

    # Remove None values
    inputs = {k: v for k, v in inputs.items() if v is not None}
    return inputs

In [9]:
import re
import json
import os
import string
from collections import Counter
from tqdm import tqdm
import more_itertools as mit

def find_ranges(iterable):
    """Yield range of consecutive numbers."""
    for group in mit.consecutive_groups(iterable):
        group = list(group)
        if len(group) == 1:
            yield group[0]
        else:
            yield group[0], group[-1]

def process_and_convert_data(data, tokenizer, post_id_divisions, save_path='Data/explanations/', drop_abnormal=False):
    """
    Combined function that processes raw entries and converts to ERASER format in one pass.
    Also splits data into train/val/test sets.
    """
    print("Processing and converting data...")

    # Initialize outputs
    train_data = []
    val_data = []
    test_data = []
    dropped = 0

    # Create directories if saving splits
    if save_path:
        os.makedirs(save_path, exist_ok=True)
        os.makedirs(os.path.join(save_path, 'docs'), exist_ok=True)
        train_fp = open(os.path.join(save_path, 'train.jsonl'), 'w')
        val_fp = open(os.path.join(save_path, 'val.jsonl'), 'w')
        test_fp = open(os.path.join(save_path, 'test.jsonl'), 'w')

    for key, value in tqdm(data.items()):
        try:
            # Extract labels
            labels = [1 if annot["label"] in ['hatespeech', 'offensive'] else 0
                     for annot in value["annotators"]]

            # Process rationales
            rationales = value.get("rationales", [])
            aggregated_rationale = aggregate_rationales(
                rationales, labels, len(value["post_tokens"]), drop_abnormal=drop_abnormal
            )

            if aggregated_rationale is None:
                dropped += 1
                continue

            inputs = align_rationales(value['post_tokens'], aggregated_rationale, tokenizer)

            # Calculate labels
            hard_label = Counter(labels).most_common(1)[0][0]
            soft_label = sum(labels) / len(labels)

            # Determine target groups (mentioned at least 3 times)
            target_groups = [t for annot in value['annotators'] for t in annot['target']]
            filtered_targets = [k for k, v in Counter(target_groups).items() if v > 2]

            # Create processed entry
            processed_entry = {
                'post_id': key,
                'input_ids': inputs['input_ids'],
                'attention_mask': inputs['attention_mask'],
                'rationales': inputs['rationales'],
                'raw_text': " ".join(value['post_tokens']),
                'hard_label': hard_label,
                'soft_label': soft_label,
                'target_groups': filtered_targets
            }

            # Convert to ERASER format if it's hateful/offensive content
            if hard_label == 1 and save_path:
                input_ids_list = inputs['input_ids'].squeeze().tolist()
                rationales_list = inputs['rationales'].squeeze().ceil().int().tolist()

                # Build evidences
                evidences = []
                indexes = sorted([i for i, each in enumerate(rationales_list) if each == 1])
                for span in find_ranges(indexes):
                    if isinstance(span, int):
                        start, end = span, span + 1
                    else:
                        start, end = span[0], span[1] + 1

                    evidences.append({
                        "docid": key,
                        "end_sentence": -1,
                        "end_token": end,
                        "start_sentence": -1,
                        "start_token": start,
                        "text": ' '.join([str(x) for x in input_ids_list[start:end]])
                    })

                eraser_entry = {
                    'annotation_id': key,
                    'classification': hard_label,
                    'evidences': [evidences],
                    'query': "What is the class?",
                    'query_type': None
                }

                # Save document
                with open(os.path.join(save_path, 'docs', key), 'w') as fp:
                    fp.write(' '.join([str(x) for x in input_ids_list]))

                # Write to appropriate split
                if key in post_id_divisions['train']:
                    train_fp.write(json.dumps(eraser_entry) + '\n')
                elif key in post_id_divisions['val']:
                    val_fp.write(json.dumps(eraser_entry) + '\n')
                elif key in post_id_divisions['test']:
                    test_fp.write(json.dumps(eraser_entry) + '\n')

            # Add to appropriate split list
            if key in post_id_divisions['train']:
                train_data.append(processed_entry)
            elif key in post_id_divisions['val']:
                val_data.append(processed_entry)
            elif key in post_id_divisions['test']:
                test_data.append(processed_entry)

        except Exception as e:
            dropped += 1
            print(f"Error processing {key}: {e}")

    if save_path:
        train_fp.close()
        val_fp.close()
        test_fp.close()

    print(f"Train: {len(train_data)}, Val: {len(val_data)}, Test: {len(test_data)}, Dropped: {dropped}")

    return {
        'train': train_data,
        'val': val_data,
        'test': test_data
    }

In [10]:
with open(data_path, 'r') as file:
    data = json.load(file)

with open('Data/post_id_divisions.json') as file:
    post_id_divisions = json.load(file)

# Process everything in one pass
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
splits = process_and_convert_data(
    data=data,
    tokenizer=tokenizer,
    post_id_divisions=post_id_divisions,
    save_path='Data/explanations/',
    drop_abnormal=False
)

# Access splits directly
train_data = splits['train']
val_data = splits['val']
test_data = splits['test']

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

 36%|███▋      | 7331/20148 [00:25<00:25, 499.07it/s]

 89%|████████▉ | 17979/20148 [00:42<00:03, 620.98it/s]

Error processing 24439295_gab: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.


100%|██████████| 20148/20148 [00:46<00:00, 437.05it/s]

Train: 15382, Val: 1922, Test: 1924, Dropped: 1


In [11]:
from HateDataset import HateDataset

# Create datasets with pre-tokenized data
train_dataset = HateDataset(data=train_data)
val_dataset = HateDataset(data=val_data)
test_dataset = HateDataset(data=test_data)

In [12]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False) # Use shuffle=False for validation
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False) # Use shuffle=False for testing

In [13]:
from HateClassifier import HateClassifier
model = HateClassifier(config)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [14]:
history = model.train(train_dataloader=train_loader, val_dataloader=val_loader)

Training on device: cuda
Model: distilbert-base-uncased
Epochs: 2
Batch size: 32
Gradient accumulation steps: 1
Effective batch size: 32
Learning rate: 1e-05
Mixed precision (AMP): True
Gradient clipping: 1.0

Epoch 1/2


Evaluating: 100%|██████████| 61/61 [00:01<00:00, 31.82batch/s]



Epoch 1 Summary:
  Train Loss: 0.5140
  Val Loss:   0.4677
  Val Acc:    0.7711
  Val F1:     0.7647
  ✓ New best model saved! (F1: 0.7647)

Epoch 2/2


Evaluating: 100%|██████████| 61/61 [00:01<00:00, 32.81batch/s]



Epoch 2 Summary:
  Train Loss: 0.4179
  Val Loss:   0.4680
  Val Acc:    0.7695
  Val F1:     0.7633

Training completed!
Best F1 Score: 0.7647
Training history saved to: ./checkpoints/training_history.json


# Evaluation

In [15]:
import pickle

result = model.predict(test_dataloader=test_loader, return_attentions=True)

# Save the result to a file
with open('prediction_results.pkl', 'wb') as f:
  pickle.dump(result, f)

print("Results saved to prediction_results.pkl")

Running inference on 61 batches...


Testing: 100%|██████████| 61/61 [00:01<00:00, 31.52batch/s]


Test Results:
  Test Loss:     0.4493
  Test Accuracy: 0.7968
  Test F1:       0.7902
Results saved to prediction_results.pkl


## Bias

In [16]:
def get_bias_evaluation_samples(data, method, group):
    """
    Get positive and negative sample IDs for bias evaluation based on method and group

    Args:
        data: list of data entries
        method: Bias evaluation method ('subgroup', 'bpsn', or 'bnsp')
        group: Target group to evaluate

    Returns:
        Tuple of (positive_ids, negative_ids)
    """
    positive_ids = []
    negative_ids = []

    for idx, row in enumerate(data):
        target_groups = row['target_groups']
        if target_groups is None:
            continue

        is_in_group = group in target_groups

        # Convert various label formats to binary toxic/non-toxic
        if 'hard_label' in row:
            is_toxic = row['hard_label'] == 1
        else:
            continue

        if method == 'subgroup':
            # Only consider samples mentioning the group
            if is_in_group:
                if is_toxic:
                    positive_ids.append(idx)
                else:
                    negative_ids.append(idx)

        elif method == 'bpsn':
            # Compare non-toxic posts mentioning the group with toxic posts NOT mentioning the group
            if is_in_group and not is_toxic:
                negative_ids.append(idx)
            elif not is_in_group and is_toxic:
                positive_ids.append(idx)

        elif method == 'bnsp':
            # Compare toxic posts mentioning the group with non-toxic posts NOT mentioning the group
            if is_in_group and is_toxic:
                positive_ids.append(idx)
            elif not is_in_group and not is_toxic:
                negative_ids.append(idx)

    return positive_ids, negative_ids

In [17]:
from collections import defaultdict
from sklearn.metrics import roc_auc_score

def calculate_gmb_metrics(
    test_data: List[Dict[str, Any]],
    probabilities: np.ndarray,
    target_groups: List[str]
):
    """
    Calculate GMB (Generalized Mean of Bias) AUC metrics from model predictions

    Args:
        probabilities: Model's probability outputs
        test_data: List of test data entries
        target_groups: List of target groups to evaluate

    Returns:
        Dictionary with GMB metrics
    """
    # Create mappings from post_id to predictions and ground truth
    prediction_scores = defaultdict(lambda: defaultdict(dict))
    ground_truth = {}

    for idx, row in enumerate(test_data):
        prediction_scores[idx] = probabilities[idx, 1]
        ground_truth[idx] = row['hard_label']

    # Calculate metrics for each target group and method
    bias_metrics = {}
    methods = ['subgroup', 'bpsn', 'bnsp']

    for method in methods:
        bias_metrics[method] = {}  # Initialize nested dictionary for each method
        for group in target_groups:
            # Get positive and negative samples based on the method
            positive_ids, negative_ids = get_bias_evaluation_samples(test_data, method, group)

            if len(positive_ids) == 0 or len(negative_ids) == 0:
                print(f"Skipping {method} for group {group}: no samples found")
                continue  # Skip if no samples for this group/method

            # Collect ground truth and predictions
            y_true = []
            y_score = []

            for post_id in positive_ids:
                if post_id in ground_truth and post_id in prediction_scores:
                    y_true.append(ground_truth[post_id])
                    y_score.append(prediction_scores[post_id])

            for post_id in negative_ids:
                if post_id in ground_truth and post_id in prediction_scores:
                    y_true.append(ground_truth[post_id])
                    y_score.append(prediction_scores[post_id])

            # Calculate AUC if we have enough samples with both classes
            if len(y_true) > 10 and len(set(y_true)) > 1:
                try:
                    auc = roc_auc_score(y_true, y_score)
                    bias_metrics[method][group] = auc
                except ValueError:
                    print(f"Could not compute AUC for {method} and group {group} due to ValueError")
                    pass

    # Calculate GMB for each method
    gmb_metrics = {}
    power = -5  # Power parameter for generalized mean

    for method in methods:
        if not bias_metrics[method]:
            continue

        scores = list(bias_metrics[method].values())
        if not scores:
            continue

        # Calculate generalized mean with p=-5
        power_mean = np.mean([score ** power for score in scores]) ** (1/power)
        gmb_metrics[f'GMB-{method.upper()}-AUC'] = power_mean

    # Calculate a combined GMB score that includes all methods
    all_scores = []
    for method in methods:
        all_scores.extend(list(bias_metrics[method].values()))

    if all_scores:
        gmb_metrics['GMB-COMBINED-AUC'] = np.mean([score ** power for score in all_scores]) ** (1/power)

    return gmb_metrics, bias_metrics

In [18]:
from itertools import chain

all_target_groups = list(chain.from_iterable(d['target_groups'] for d in train_data + val_data + test_data))

In [19]:
from collections import Counter
# Get top 10 most common target groups in the full dataset

# Remove None
all_target_groups = [group for group in all_target_groups if group != 'None' and group != 'Other']
counter = Counter(all_target_groups)

n_common = 10
bias_target_groups = [tg[0] for tg in counter.most_common(n_common)]

In [20]:
gmb_metrics, bias_details = calculate_gmb_metrics(
  test_data=test_data,
  probabilities=result['probabilities'],
  target_groups=bias_target_groups
)

In [21]:
print('GMB-Metrics')
for key, value in gmb_metrics.items():
  print(f'{key}: {value}')

GMB-Metrics
GMB-SUBGROUP-AUC: 0.8081792990282538
GMB-BPSN-AUC: 0.7716453586649427
GMB-BNSP-AUC: 0.8105021011920741
GMB-COMBINED-AUC: 0.7955413062110042


In [22]:
print('Bias Details')
print()
for key, entry in bias_details.items():
  print(f"Metrics: {key}")
  for subgroup, value in entry.items():
    print(f'{subgroup}: {value}')
  print()

Bias Details

Metrics: subgroup
African: 0.8964992389649924
Jewish: 0.8120300751879699
Islam: 0.9719626168224299
Homosexual: 0.8660714285714286
Women: 0.6092896174863388
Refugee: 0.7757575757575758
Arab: 0.896551724137931
Hispanic: 1.0
Asian: 1.0
Caucasian: 0.8666666666666667

Metrics: bpsn
African: 0.6941133983387504
Jewish: 0.5944003964321111
Islam: 0.9379227053140097
Homosexual: 0.8585760517799352
Women: 0.7906259636139378
Refugee: 0.915990381725278
Arab: 0.7466307277628033
Hispanic: 0.868515205724508
Asian: 0.9844858156028369
Caucasian: 0.9733806566104704

Metrics: bnsp
African: 0.9400208674040598
Jewish: 0.9465324622610511
Islam: 0.8941330514379069
Homosexual: 0.8827938749312305
Women: 0.7830403920905864
Refugee: 0.7310062818537395
Arab: 0.941630977085081
Hispanic: 0.9777262057191635
Asian: 0.8748856777025791
Caucasian: 0.634957264957265



# XAI

In [162]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from typing import List, Dict, Tuple
from sklearn.metrics import precision_recall_curve, auc, f1_score, precision_score, recall_score

class FaithfulnessMetrics:
  """
  Compute faithfulness metrics using the model's existing predict() method.
  Creates modified datasets and uses DataLoader for efficient batched processing.
  """

  def __init__(self, model, tokenizer, dataset_class, batch_size=32):
      self.model = model
      self.tokenizer = tokenizer
      self.dataset_class = dataset_class
      self.batch_size = batch_size

      # Get special token IDs
      self.special_token_ids = {
          tokenizer.cls_token_id,
          tokenizer.sep_token_id,
      }
      self.special_token_ids = {x for x in self.special_token_ids if x is not None}

  def compute_all_metrics(
      self,
      test_data: List[Dict],  # Your original test data
      test_results: List[Dict], # Results from prediction
      k: int = 5  # Number of top tokens to consider
  ) -> Dict[str, float]:
      """
      Compute all ERASER metrics efficiently using DataLoader approach

      Args:
          test_data: List of test instances (each with input_ids, attention_mask, rationales, labels)
          test_results: List of dictionaries containing attention scores for each instance

      Returns:
          Dictionary with all metrics
      """
      print("Computing ERASER metrics using DataLoader approach...")

      # Extract lists for easier processing
      input_ids_list = [item['input_ids'] for item in test_data]
      attention_masks_list = [item['attention_mask'] for item in test_data]
      human_rationales = [item['rationales'] for item in test_data]
      attention_scores = [item for item in test_results['attentions']]

      # 1. Extract top-k as hard predictions
      hard_predictions = self._extract_top_k_tokens(
          attention_scores, attention_masks_list, input_ids_list, k
      )

      # 2. PLAUSIBILITY METRICS
      print("\n[1/3] Computing plausibility metrics...")
      auprc = self._compute_auprc(
          attention_scores, human_rationales, attention_masks_list, input_ids_list
      )
      token_f1, token_prec, token_rec = self._compute_token_f1(
          hard_predictions, human_rationales, attention_masks_list
      )

      # 3. FAITHFULNESS METRICS
      print("[2/3] Computing comprehensiveness scores...")
      comprehensiveness_scores = self._compute_comprehensiveness(
          test_data, test_results, hard_predictions
      )

      print("[3/3] Computing sufficiency scores...")
      sufficiency_scores = self._compute_sufficiency(
          test_data, test_results, hard_predictions
      )

      return {
          # Plausibility
          'auprc': auprc,
          'token_f1': token_f1,
          'token_precision': token_prec,
          'token_recall': token_rec,

          # Faithfulness
          'comprehensiveness': np.mean(comprehensiveness_scores),
          'sufficiency': np.mean(sufficiency_scores),

          # Additional
          'avg_rationale_length': k,
      }

  def _calculate_average_rationale_length(
      self,
      human_rationales: List[torch.Tensor],
      attention_masks_list: List[torch.Tensor],
      input_ids_list: List[torch.Tensor]
  ) -> int:
      """Calculate average number of content rationale tokens"""
      lengths = []
      for idx, (rat, mask) in enumerate(zip(human_rationales, attention_masks_list)):
          valid_positions = mask.bool().cpu().numpy().flatten()

          # Exclude special tokens
          input_ids = input_ids_list[idx].cpu().numpy().flatten()
          is_special = np.isin(input_ids, list(self.special_token_ids))
          content_positions = valid_positions & ~is_special

          rat_count = (rat.cpu().numpy().flatten()[content_positions] == 1).sum()
          lengths.append(rat_count)

      return max(1, int(np.mean(lengths)))

  def _extract_top_k_tokens(
      self,
      attention_scores: List[np.ndarray],
      attention_masks_list: List[torch.Tensor],
      input_ids_list: List[torch.Tensor],
      k: int
  ) -> List[np.ndarray]:
      """Extract top-k content tokens as hard predictions"""
      hard_predictions = []

      for idx, (attn, mask) in enumerate(zip(attention_scores, attention_masks_list)):
          pred_mask = np.zeros_like(attn, dtype=int)
          valid_positions = mask.bool().cpu().numpy().flatten()

          # Exclude special tokens
          input_ids = input_ids_list[idx].cpu().numpy().flatten()
          is_special = np.isin(input_ids, list(self.special_token_ids))
          content_positions = valid_positions & ~is_special

          content_attn = attn[content_positions]

          if k > 0 and len(content_attn) > 0:
              k_actual = min(k, len(content_attn))
              top_k_within_content = np.argsort(content_attn)[-k_actual:]
              content_indices = np.where(content_positions)[0]
              top_k_indices = content_indices[top_k_within_content]
              pred_mask[top_k_indices] = 1

          hard_predictions.append(pred_mask)

      return hard_predictions

  def _compute_auprc(
      self,
      attention_scores: List[np.ndarray],
      human_rationales: List[torch.Tensor],
      attention_masks_list: List[torch.Tensor],
      input_ids_list: List[torch.Tensor]
  ) -> float:
      """Compute AUPRC for soft attention scores"""
      all_scores = []
      all_labels = []

      for idx, (attn, rat, mask) in enumerate(zip(attention_scores, human_rationales, attention_masks_list)):
          valid_positions = mask.bool().cpu().numpy().flatten()

          # Exclude special tokens
          input_ids = input_ids_list[idx].cpu().numpy().flatten()
          is_special = np.isin(input_ids, list(self.special_token_ids))
          content_positions = valid_positions & ~is_special

          all_scores.extend(attn[content_positions].tolist())
          all_labels.extend(rat.cpu().numpy().flatten()[content_positions].astype(int).tolist())

      all_scores = np.array(all_scores, dtype=float)
      all_labels = np.array(all_labels, dtype=int)

      if len(np.unique(all_labels)) < 2:
          print(f"Warning: Only one class in labels: {np.unique(all_labels)}")
          return 0.0

      precision, recall, _ = precision_recall_curve(all_labels, all_scores)
      return auc(recall, precision)

  def _compute_token_f1(
      self,
      hard_predictions: List[np.ndarray],
      human_rationales: List[torch.Tensor],
      attention_masks_list: List[torch.Tensor]
  ) -> Tuple[float, float, float]:
      """Compute token-level F1, Precision, Recall"""
      all_preds = []
      all_labels = []

      for pred, rat, mask in zip(hard_predictions, human_rationales, attention_masks_list):
          valid_positions = mask.bool().cpu().numpy().flatten()
          all_preds.extend(pred[valid_positions].astype(int).tolist())
          all_labels.extend(rat.cpu().numpy().flatten()[valid_positions].astype(int).tolist())

      all_preds = np.array(all_preds, dtype=int)
      all_labels = np.array(all_labels, dtype=int)

      f1 = f1_score(all_labels, all_preds, zero_division=0)
      precision = precision_score(all_labels, all_preds, zero_division=0)
      recall = recall_score(all_labels, all_preds, zero_division=0)

      return f1, precision, recall

  def _compute_comprehensiveness(
      self,
      test_data: List[Dict],
      test_results: List[Dict],
      hard_predictions: List[np.ndarray]
  ) -> List[float]:
      """
      Compute comprehensiveness: how much does REMOVING rationales hurt?
      Uses DataLoader approach for efficiency
      """
      # Create modified dataset (remove rationales from attention mask)
      modified_data = []
      for item, rationale_mask in zip(test_data, hard_predictions):
          modified_item = self._create_comprehensiveness_instance(item, rationale_mask)
          modified_data.append(modified_item)

      # Create DataLoader
      modified_dataset = self.dataset_class(modified_data)
      modified_loader = DataLoader(
          modified_dataset,
          batch_size=self.batch_size,
          shuffle=False
      )

      # Get predictions using model's predict method
      results = self.model.predict(modified_loader, return_attentions=False)
      modified_probs = results['probabilities']

      # Calculate comprehensiveness scores
      comprehensiveness_scores = []
      for idx, (prob, td) in enumerate(zip(test_results['probabilities'], test_data)):
          label = int(td['hard_label'])
          original_prob = prob[label] # Probability from normal prediction process for the label
          modified_prob = modified_probs[idx][label]

          # Comprehensiveness = original - modified (higher is better)
          comp_score = original_prob - modified_prob
          comprehensiveness_scores.append(comp_score)

      return comprehensiveness_scores

  def _compute_sufficiency(
      self,
      test_data: List[Dict],
      test_results: List[Dict],
      hard_predictions: List[np.ndarray]
  ) -> List[float]:
      """
      Compute sufficiency: how well do ONLY rationales predict?
      Uses DataLoader approach for efficiency
      """
      # Create modified dataset (keep only rationales in attention mask)
      modified_data = []
      for item, rationale_mask in zip(test_data, hard_predictions):
          modified_item = self._create_sufficiency_instance(item, rationale_mask)
          modified_data.append(modified_item)

      # Create DataLoader
      modified_dataset = self.dataset_class(modified_data)
      modified_loader = DataLoader(
          modified_dataset,
          batch_size=self.batch_size,
          shuffle=False
      )

      # Get predictions using model's predict method
      results = self.model.predict(modified_loader, return_attentions=False)
      modified_probs = results['probabilities']

      # Calculate sufficiency scores
      sufficiency_scores = []
      for idx, (prob, td) in enumerate(zip(test_results['probabilities'], test_data)):
          label = int(td['hard_label'])
          original_prob = prob[label] # Probability from normal prediction process for the label
          modified_prob = modified_probs[idx][label]

          # Sufficiency = original - modified (lower/negative is better)
          suff_score = original_prob - modified_prob
          sufficiency_scores.append(suff_score)

      return sufficiency_scores

  def _create_comprehensiveness_instance(
      self,
      item: Dict,
      rationale_mask: np.ndarray
  ) -> Dict:
      """
      Create instance for comprehensiveness: REMOVE rationales from attention
      Keep: CLS + non-rationale content tokens + SEP
      """
      input_ids = item['input_ids'].cpu().numpy().flatten()
      orig_mask = item['attention_mask'].cpu().numpy().flatten()

      # Start with original mask
      new_mask = orig_mask.copy()

      # Zero out rationale positions (except CLS and SEP)
      for i in range(len(new_mask)):
          if rationale_mask[i] == 1:  # This is a rationale
              # Don't mask if it's CLS or SEP
              if input_ids[i] not in self.special_token_ids:
                  new_mask[i] = 0

      return {
          'input_ids': torch.tensor(input_ids).unsqueeze(0),
          'attention_mask': torch.tensor(new_mask).unsqueeze(0),
          'rationales': item['rationales'],
          'hard_label': item['hard_label'],
      }

  def _create_sufficiency_instance(
      self,
      item: Dict,
      rationale_mask: np.ndarray
  ) -> Dict:
      """
      Create instance for sufficiency: Keep ONLY rationales in attention
      Keep: CLS + rationale tokens + SEP
      """
      input_ids = item['input_ids'].cpu().numpy().flatten()
      orig_mask = item['attention_mask'].cpu().numpy().flatten()

      # Start with zeros
      new_mask = np.zeros_like(orig_mask)

      # Always keep CLS and SEP
      for i in range(len(new_mask)):
          if input_ids[i] in self.special_token_ids:
              new_mask[i] = 1

      # Keep rationale positions
      for i in range(len(new_mask)):
          if rationale_mask[i] == 1 and orig_mask[i] == 1:
              new_mask[i] = 1

      return {
          'input_ids': torch.tensor(input_ids).unsqueeze(0),
          'attention_mask': torch.tensor(new_mask).unsqueeze(0),
          'rationales': item['rationales'],
          'hard_label': item['hard_label'],
      }

In [163]:
calculator = FaithfulnessMetrics(
    model=model,
    tokenizer=tokenizer,
    dataset_class=HateDataset,
    batch_size=32
)

In [164]:
input_ids_list = [item['input_ids'] for item in test_data]
attention_masks_list = [item['attention_mask'] for item in test_data]
human_rationales = [item['rationales'] for item in test_data]
attention_scores = [item for item in result['attentions']]
k = 5

In [165]:
hard_predictions = calculator._extract_top_k_tokens(
  attention_scores,
  attention_masks_list,
  input_ids_list,
  k
)

In [166]:
sufficiency_scores = calculator._compute_sufficiency(test_data, result, hard_predictions)

Running inference on 61 batches...


Testing: 100%|██████████| 61/61 [00:01<00:00, 30.83batch/s]


Test Results:
  Test Loss:     0.5752
  Test Accuracy: 0.7173
  Test F1:       0.7145


In [167]:
comprehensiveness_scores = calculator._compute_comprehensiveness(test_data, result, hard_predictions)

Running inference on 61 batches...


Testing: 100%|██████████| 61/61 [00:01<00:00, 31.74batch/s]


Test Results:
  Test Loss:     0.8646
  Test Accuracy: 0.5473
  Test F1:       0.5328


In [168]:
print(f"Comprehensiveness: {np.mean(comprehensiveness_scores)}")
print(f"Sufficiency: {np.mean(sufficiency_scores)}")

Comprehensiveness: 0.19652852416038513
Sufficiency: 0.054969824850559235


In [169]:
xai_results = calculator.compute_all_metrics(test_data, result, k)

Computing ERASER metrics using DataLoader approach...

[1/3] Computing plausibility metrics...
[2/3] Computing comprehensiveness scores...
Running inference on 61 batches...


Testing: 100%|██████████| 61/61 [00:01<00:00, 31.77batch/s]



Test Results:
  Test Loss:     0.8646
  Test Accuracy: 0.5473
  Test F1:       0.5328
[3/3] Computing sufficiency scores...
Running inference on 61 batches...


Testing: 100%|██████████| 61/61 [00:01<00:00, 31.37batch/s]


Test Results:
  Test Loss:     0.5752
  Test Accuracy: 0.7173
  Test F1:       0.7145


In [170]:
xai_results

{'auprc': np.float64(0.2183696732701997),
 'token_f1': 0.3020481346114061,
 'token_precision': 0.2280062467464862,
 'token_recall': 0.44730392156862747,
 'comprehensiveness': np.float32(0.19652852),
 'sufficiency': np.float32(0.054969825),
 'avg_rationale_length': 5}

In [176]:
# Skip all entries that are normal (label 0) in test data
test_data_hate_only = []
test_results_hate_only = {'attentions': [], 'probabilities': []}
for idx, td in enumerate(test_data):
  if td['hard_label'] == 1:
    test_data_hate_only.append(td)
    test_results_hate_only['attentions'].append(result['attentions'][idx])
    test_results_hate_only['probabilities'].append(result['probabilities'][idx])

In [177]:
xai_results_hate_only = calculator.compute_all_metrics(test_data_hate_only, test_results_hate_only, k)

Computing ERASER metrics using DataLoader approach...

[1/3] Computing plausibility metrics...
[2/3] Computing comprehensiveness scores...
Running inference on 36 batches...


Testing: 100%|██████████| 36/36 [00:01<00:00, 29.67batch/s]



Test Results:
  Test Loss:     1.1826
  Test Accuracy: 0.3126
  Test F1:       0.2382
[3/3] Computing sufficiency scores...
Running inference on 36 batches...


Testing: 100%|██████████| 36/36 [00:01<00:00, 31.06batch/s]


Test Results:
  Test Loss:     0.5711
  Test Accuracy: 0.6874
  Test F1:       0.4074


In [178]:
xai_results_hate_only

{'auprc': np.float64(0.37660807521899525),
 'token_f1': 0.41316856900292426,
 'token_precision': 0.3838737949167397,
 'token_recall': 0.44730392156862747,
 'comprehensiveness': np.float32(0.3761324),
 'sufficiency': np.float32(0.09439625),
 'avg_rationale_length': 5}